In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

import seaborn as sns
import geopandas as gpd
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
explanatory_file_path = "data/7_training_data/explanatory_df.csv"
explanatory_df = pd.read_csv(explanatory_file_path, index_col=0)
explanatory_df.head()


,lat_x,lon_x,date,LIA,VH,VV,VVVH_ratio,angle,sm_1,sm_3,...,system:index,lat_y,lon_y,elevation,aspect,slope,land_cov,canopy_height,gldas_mean,gldas_stddev
id,,,,,,,,,,,,,,,,,,,,,
121_APC_D15,NaN,NaN,2020-12-02,37.158626,-13.377886,-6.305714,0.188173,39.002327,36.214001,36.373625,...,37909,NaN,NaN,11,270.0,1.858527,12,11,35.598789,3.22416
121_APC_D15,NaN,NaN,2021-09-14,37.852636,-12.313376,-4.441834,0.300894,36.006908,41.771999,37.499333,...,37909,NaN,NaN,11,270.0,1.858527,12,11,35.598789,3.22416
121_APC_D15,NaN,NaN,2021-12-08,37.155728,-14.935464,-7.701659,0.137663,38.999432,38.425999,38.001083,...,37909,NaN,NaN,11,270.0,1.858527,12,11,35.598789,3.22416
121_APC_D15,NaN,NaN,2022-06-10,37.159135,-14.816424,-6.980119,0.167454,39.002838,35.361000,35.660459,...,37909,NaN,NaN,11,270.0,1.858527,12,11,35.598789,3.22416
121_APC_D15,NaN,NaN,2021-09-23,37.158385,-13.969044,-5.805131,0.222621,39.002090,35.499001,35.744584,...,37909,NaN,NaN,11,270.0,1.858527,12,11,35.598789,3.22416


In [3]:
explain_vars = [
    "LIA",
    "VH",
    "VV",
    "VVVH_ratio",
    "angle",
    "sm_1",
    "sm_3",
    "sm_7",
    "sm_30",
    "precipitation",
    "prec_3",
    "prec_7",
    "prec_30",
    "system:index",
    "elevation",
    "aspect",
    "slope",
    "land_cov",
    "canopy_height",
    "gldas_mean",
    "gldas_stddev",
]


Index(['lat_x', 'lon_x', 'date', 'LIA', 'VH', 'VV', 'VVVH_ratio', 'angle',
       'sm_1', 'sm_3', 'sm_7', 'sm_30', 'precipitation', 'prec_3', 'prec_7',
       'prec_30', 'system:index', 'lat_y', 'lon_y', 'elevation', 'aspect',
       'slope', 'land_cov', 'canopy_height', 'gldas_mean', 'gldas_stddev'],
      dtype='object')